In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# Token

In [2]:
import json
import requests as req
import base64

# Step 1. Setting AuthOption
json_file = open("auth.json")
auth = json.load(json_file)
auth_data = "{}:{}".format(
    auth['CLIENT_ID'],
    auth['CLIENT_SECRET']
)
authorization = "Basic " + \
        base64.b64encode(auth_data.encode('ascii')).decode("ascii")

# Step 2. Setting Request Datas
token_url = "https://accounts.spotify.com/api/token"
headers = {
    "Authorization": authorization
}
data = {
    "grant_type": "client_credentials"
}

# Step 3. Request
res = req.post(token_url, data=data, headers=headers)
token = res.json()['access_token']

res.json()

{'access_token': 'BQAwCO3Dxm1KZJnjcQ06yERUSqctY3GY1X2j7MlAD38-iypbMSz3Jjt7hz3OhK1KGJAcWLkZ-GFHiKMDOYU',
 'token_type': 'Bearer',
 'expires_in': 3600}

# Search

In [3]:
import requests as req
from functools import reduce
from urllib.parse import urlencode
from IPython.display import clear_output
import pandas as pd

# Step 1. Setting Request Datas
search_url = "https://api.spotify.com/v1/search"
query = urlencode({
    "q": "10cm 서랍",
    "type": "track",
    "market": "KR"
})
headers = {
    "Authorization": "Bearer {}".format(token)
}

res = req.get("{}?{}".format(search_url, query), headers=headers)

### Search Program

In [59]:
sel_tracks = pd.DataFrame(
        columns=['trackId', 'trackName', 'artistIds', 'artistNames', "image"])

while True:
    q = input("검색어를 입력해주세요.")
    offset = 0
    limit = 10
    
    while True:
        search_url = "https://api.spotify.com/v1/search"
        query = urlencode({
            "q": q,
            "type": "track",
            "market": "KR",
            "limit": limit,
            "offset": offset
        })
        headers = {
            "Authorization": "Bearer {}".format(token)
        }
        res = req.get("{}?{}".format(search_url, query), headers=headers)
        result = res.json()['tracks']
        _result = list()
        
        for item in result['items']:
            _id = item["id"]
            _name = item["name"]
            album = item['album']
            images = album["images"]

            if len(images) == 0:
                _image = ""
            else:
                _image = images[0]['url']

            artist_list = album['artists']
            artists = reduce(
                lambda acc, cur: cur[1]['name'] if cur[0] == 0 else acc +
                "," + cur[1]['name'],
                enumerate(artist_list),
                ""
            )
            artists_id = reduce(
                lambda acc, cur: cur[1]['id'] if cur[0] == 0 else acc +
                "," + cur[1]['id'],
                enumerate(artist_list),
                ""
            )
            _result.append({
                "trackId": _id,
                "trackName": _name,
                "artistIds": artists_id,
                "artistNames": artists,
                "image": _image
            })
        
        for idx, _ in enumerate(_result):
            print("\t{}.{} - {} ({})".format(idx + 1, _['artistNames'],
                                             _['trackName'], _['trackId']))
        _total = (offset + 1)
        _total = result['total'] if  _total > result['total'] else _total
        print("{}/{}".format(_total, result['total']))
        _action = input("선택은 번호입력을 해주세요." +
                            "\n종료는 exit, 다음페이지는 next를 입력해주세요.")
        
        if _action == "exit":
            clear_output()
            break
        elif _action == "next":
            offset += limit
            clear_output()
        else:
            _action = int(_action) - 1
            sel_tracks = sel_tracks.append(
                _result[_action], ignore_index=True)
            clear_output()
            break
    
    _action = input("계속하시려면 아무거나, 종료는 exit를 입력해주세요.")

    if _action == "exit":
        break
    else:
        clear_output(wait=True)
        
sel_tracks.head()

계속하시려면 아무거나, 종료는 exit를 입력해주세요.exit


,trackId,trackName,artistIds,artistNames,image
0,2FWquqPNxte8iqZ3ATQG0p,All I Wanna Do (K) (Feat. Hoody & Loco),4XDi67ZENZcbfKnvMnTYsI,Jay Park,https://i.scdn.co/image/ab67616d0000b27382ecc5...
1,4NKQTbmiBtm1MwqBbQQhpM,DRIVE (Feat. GRAY),4XDi67ZENZcbfKnvMnTYsI,Jay Park,https://i.scdn.co/image/ab67616d0000b27382ecc5...


# Artists

In [60]:
import pandas as pd
import requests as req
from urllib.parse import urlencode
import math as mt
from functools import reduce

# 1. Data Parsing
ids = reduce(
    lambda acc, cur: acc + cur + ",",sel_tracks['artistIds'].values, "")[:-1]

# 2. Data Preprocessing (중복 가수명 제거)
ids = list(set(ids.split(",")))
ids = reduce(
    lambda acc, cur: acc + cur + ",",ids, "")[:-1]

# 3. Setting Request Datas
artists_url = "https://api.spotify.com/v1/artists"
query = urlencode({
    "ids": ids
})
headers = {
    "Authorization": "Bearer {}".format(token)
}
res = req.get("{}?{}".format(artists_url, query), headers=headers)

## 아이고 나 바빠 ㅠㅠㅠ

In [61]:
from src.Spotify.get_genres import get_genres

genres = get_genres(token, sel_tracks)
genres.head()

,artistIds,genres
0,4XDi67ZENZcbfKnvMnTYsI,"k-pop,pop,hip-hop"


# Audio Features

In [62]:
import pandas as pd
import requests as req
from urllib.parse import urlencode
import math as mt

# 1. Data Parsing
ids = reduce(
    lambda acc, cur: acc + cur + ",",sel_tracks['trackId'].values, "")[:-1]

# 2. Setting Request Datas
features_url = "https://api.spotify.com/v1/audio-features"
query = urlencode({
    "ids": ids
})
headers = {
    "authorization": "Bearer {}".format(token)
}
res = req.get("{}?{}".format(features_url, query), headers=headers)

## 아이고 나 바빠 ㅠㅠㅠ

In [63]:
from src.Spotify.get_features import get_features

features = get_features(token, sel_tracks)
features

,trackId,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2FWquqPNxte8iqZ3ATQG0p,0.403,0.724,8,-6.064,0.246,0.377,0,0.0927,0.428,97.133
1,4NKQTbmiBtm1MwqBbQQhpM,0.650,0.749,5,-3.707,0.041,0.186,0,0.0723,0.929,150.026


# Recommendations

In [64]:
_sel_tracks = pd.DataFrame()

for idx, row in sel_tracks.iterrows():
    artist_ids = row['artistIds'].split(",")
    artist_names = row['artistNames'].split(",")
    if len(artist_ids) > 1:
        for _idx, artist_id in enumerate(artist_ids):
            _row = row.copy()
            artist_name = artist_names[_idx]
            _row['artistIds'] = artist_id
            _row['artistNames'] = artist_name
            _sel_tracks = _sel_tracks.append(_row,
                                        ignore_index=True)
    else:
        _sel_tracks = _sel_tracks.append(row,
                                        ignore_index=True)
        
        
_sel_tracks

,artistIds,artistNames,image,trackId,trackName
0,4XDi67ZENZcbfKnvMnTYsI,Jay Park,https://i.scdn.co/image/ab67616d0000b27382ecc5...,2FWquqPNxte8iqZ3ATQG0p,All I Wanna Do (K) (Feat. Hoody & Loco)
1,4XDi67ZENZcbfKnvMnTYsI,Jay Park,https://i.scdn.co/image/ab67616d0000b27382ecc5...,4NKQTbmiBtm1MwqBbQQhpM,DRIVE (Feat. GRAY)


In [75]:
merges = pd.merge(_sel_tracks, genres, on="artistIds")
merges = pd.merge(merges, features, on="trackId")
merges.rename({
    "trackId": "seed_tracks",
    "artistIds": "seed_artists",
    "genres": "seed_genres",
}, axis=1, inplace=True)
feature_cols = ['seed_tracks', 'seed_artists', 'seed_genres',
                'acousticness', 'danceability', 'energy', 
                'instrumentalness','key', 'liveness', 'loudness', 
                'speechiness', 'tempo', 'valence']
merges = merges[feature_cols]

merges.head(10)

,seed_tracks,seed_artists,seed_genres,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
0,2FWquqPNxte8iqZ3ATQG0p,4XDi67ZENZcbfKnvMnTYsI,"k-pop,pop,hip-hop",0.377,0.403,0.724,0,8,0.0927,-6.064,0.246,97.133,0.428
1,4NKQTbmiBtm1MwqBbQQhpM,4XDi67ZENZcbfKnvMnTYsI,"k-pop,pop,hip-hop",0.186,0.650,0.749,0,5,0.0723,-3.707,0.041,150.026,0.929


In [79]:
# 1. Setting Request Datas
reco_uri = "https://api.spotify.com/v1/recommendations"

for idx in range(len(merges)):
    query = merges.iloc[idx].to_dict()
    query['market'] = 'KR'
    query['limit'] = 100

    query = urlencode(query)
    headers = {
        "authorization": "Bearer {}".format(token)
    }
    res = req.get("{}?{}".format(reco_uri, query), headers=headers)

## 아이고 나 바빠 ㅠㅠㅠ

In [83]:
from src.Spotify.get_recommend import get_recommend

reco_tracks = get_recommend(sel_tracks, features, genres, token)
reco_tracks.head()

,trackId,trackName,artistIds,artistName,image
0,4pjywwBkyhW26sow2ULVoW,April Sick,5i8AqrrtbwtPqzkavePQy1,lalasweet,https://i.scdn.co/image/ab67616d0000b27348f615...
1,6WbsZWuHAwF0TfKWTWIU1k,WON & ONLY (feat.Jay Park),57W9ikVc6O2wLDtmclSjvN,Simon Dominic,https://i.scdn.co/image/ab67616d0000b273073efb...
2,5NSPsOGVNWYOMeeyZPF7HY,Overdose,6wPBIyIIMxoqgux29bGF8I,EXO-K,https://i.scdn.co/image/ab67616d0000b27332d9eb...
3,0LpZo1lRGyuiEyb5CtF3ja,Feeling,4Kxlr1PRlDKEB0ekOCyHgX,BIGBANG,https://i.scdn.co/image/ab67616d0000b273ca89c0...
4,4bNqSJ92afDvDZiYgpckVf,Don't Be so Shy - Filatov & Karas Remix,74eY8wbrhhVD7pACbBHwHw,Imany,https://i.scdn.co/image/ab67616d0000b2739fa97b...
